In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("seismic-bumps.csv")
data.head()

,seismic,seismoacoustic,shift,genergy,gpuls,gdenergy,gdpuls,hazard,nbumps,nbumps2,nbumps3,nbumps4,nbumps5,nbumps6,nbumps7,nbumps89,energy,maxenergy,class
0,a,a,N,15180,48,-72,-72,a,0,0,0,0,0,0,0,0,0,0,0
1,a,a,N,14720,33,-70,-79,a,1,0,1,0,0,0,0,0,2000,2000,0
2,a,a,N,8050,30,-81,-78,a,0,0,0,0,0,0,0,0,0,0,0
3,a,a,N,28820,171,-23,40,a,1,0,1,0,0,0,0,0,3000,3000,0
4,a,a,N,12640,57,-63,-52,a,0,0,0,0,0,0,0,0,0,0,0


In [2]:
data.shape

(2584, 19)

In [3]:
data['class'].value_counts()

0    2414
1     170
Name: class, dtype: int64

In [4]:
data['seismic'].unique()

array(['a', 'b'], dtype=object)

In [5]:
data['seismic'].value_counts()

a    1682
b     902
Name: seismic, dtype: int64

In [6]:
data['seismoacoustic'].unique()

array(['a', 'b', 'c'], dtype=object)

In [7]:
data['seismoacoustic'].value_counts()

a    1580
b     956
c      48
Name: seismoacoustic, dtype: int64

In [8]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
data['seismic']= label_encoder.fit_transform(data['seismic']) 
  
data['seismic'].unique() 

array([0, 1])

In [9]:
data['seismic']

0       0
1       0
2       0
3       0
4       0
       ..
2579    1
2580    1
2581    1
2582    0
2583    0
Name: seismic, Length: 2584, dtype: int32

In [10]:
data['seismoacoustic']= label_encoder.fit_transform(data['seismoacoustic']) 
  
data['seismoacoustic'].unique() 

array([0, 1, 2])

In [11]:
data['hazard']= label_encoder.fit_transform(data['hazard']) 
  
data['hazard'].unique() 

array([0, 1, 2])

In [12]:
data['shift']= label_encoder.fit_transform(data['shift']) 
  
data['shift'].unique() 

array([0, 1])

In [14]:
data.head()

,seismic,seismoacoustic,shift,genergy,gpuls,gdenergy,gdpuls,hazard,nbumps,nbumps2,nbumps3,nbumps4,nbumps5,nbumps6,nbumps7,nbumps89,energy,maxenergy,class
0,0,0,0,15180,48,-72,-72,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,14720,33,-70,-79,0,1,0,1,0,0,0,0,0,2000,2000,0
2,0,0,0,8050,30,-81,-78,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,28820,171,-23,40,0,1,0,1,0,0,0,0,0,3000,3000,0
4,0,0,0,12640,57,-63,-52,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# to correctly set up the cross-validation
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import NeighbourhoodCleaningRule
from imblearn.over_sampling import ADASYN

In [16]:
# load data (just some cells to speed up the computations)
# we will be using KNN in our cleaning procedure so with the whole 
# dataset in my laptop it takes quite a while...

# you see already some of the limitations of these techniques ;)


data = pd.read_csv("creditcard.csv").sample(50000, random_state=0)

data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
183484,125821.0,-0.323334,1.057455,-0.048341,-0.607204,1.259821,-0.091761,1.159101,-0.124335,-0.174640,...,-0.207098,-0.433890,-0.261613,-0.046651,0.211512,0.008297,0.108494,0.161139,40.00,0
255448,157235.0,-0.349718,0.932619,0.142992,-0.657071,1.169784,-0.733369,1.009985,-0.071069,-0.302083,...,-0.271537,-0.833209,-0.030360,0.490035,-0.404816,0.134350,0.076830,0.175562,1.98,0
244749,152471.0,-1.614711,-2.406570,0.326194,0.665520,2.369268,-1.775367,-1.139049,0.329904,0.903813,...,0.701399,1.134489,0.965054,0.640981,-1.801998,-1.041114,0.286285,0.437322,96.00,0
63919,50927.0,-2.477184,0.860613,1.441850,1.051019,-1.856621,2.078384,0.510828,-0.243399,-0.260691,...,0.810408,0.692245,0.150121,-0.260777,0.005183,-0.177847,-0.510060,-0.660533,308.00,0
11475,19899.0,1.338831,-0.547264,0.737389,-0.212383,-1.110039,-0.525744,-0.801403,-0.063672,0.997276,...,-0.139436,-0.074719,0.067055,0.333122,0.379087,-0.268706,-0.002769,0.003272,5.00,0


In [17]:
# separate dataset into train and test

X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Class', axis=1) , 
    data['Class'], # just the target
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((35000, 30), (15000, 30))

In [18]:
# adasyn

adasyn = ADASYN(
        sampling_strategy='auto',  # samples only the minority class
        random_state=0,  # for reproducibility
        n_neighbors=5,
        n_jobs=4,
)

In [19]:
###################
## IMPORTANT  
##################=

# The sampling strategy needs to be set to all, or with
# a specific dictionary, because after ADASYN, our
# previous minority class is no longer minority!!

ncr = NeighbourhoodCleaningRule(
    sampling_strategy='all',# undersamples all classes
    n_neighbors=3, 
    kind_sel='mode', 
    threshold_cleaning=0.1, # the threshold to evaluate a class for cleaning (used only for clearning step)
) 

In [20]:
model = make_pipeline(
    MinMaxScaler(),
    adasyn,
    ncr, 
    RandomForestClassifier(
            n_estimators=100, random_state=39, max_depth=3, n_jobs=4
        ),
)

In [21]:
model.fit(X_train, y_train)

C:\Users\Vamshi\AppData\Local\Programs\Python\Python310\lib\site-packages\imblearn\under_sampling\_prototype_selection\_edited_nearest_neighbours.py:158: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  nnhood_label, _ = mode(nnhood_label, axis=1)
C:\Users\Vamshi\AppData\Local\Programs\Python\Python310\lib\site-packages\imblearn\under_sampling\_prototype_selection\_edited_nearest_neighbours.py:158: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will be

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('adasyn', ADASYN(n_jobs=4, random_state=0)),
                ('neighbourhoodcleaningrule',
                 NeighbourhoodCleaningRule(kind_sel='mode',
                                           sampling_strategy='all',
                                           threshold_cleaning=0.1)),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=3, n_jobs=4,
                                        random_state=39))])

In [22]:
pred_train = model.predict_proba(X_train)[:,1]
pred_test = model.predict_proba(X_test)[:,1]

In [23]:
print('Train roc-auc: {}'.format(roc_auc_score(y_train, pred_train)))


Train roc-auc: 0.9981872590065869


In [24]:
print('Test roc-auc: {}'.format(roc_auc_score(y_test, pred_test)))

Test roc-auc: 0.9914903697224804
